In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Define classes
classes = ["Bael diseased", "Basil healthy", "Jatropa diseased", "Jatropa healthy", "Lemon diseased", "Lemon healthy", "Mango diseased", "Mango healthy"]
data_dir = r"C:\Users\user7\Desktop\Crop Doctor\model 3\Dataset"

# Enhanced data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.3
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True,
    subset='training'
)

test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False,
    subset='validation'
)

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze more layers
for layer in vgg_model.layers[:8]:
    layer.trainable = False
for layer in vgg_model.layers[8:]:
    layer.trainable = True

# Add custom layers with batch normalization and L2 regularization
x = layers.Flatten()(vgg_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='fc6')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='fc7')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(len(classes), activation='softmax', name='predictions')(x)

model = models.Model(vgg_model.input, output)

# Compile the model with AdamW optimizer
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch < 50:
        return lr * tf.math.exp(-0.1)
    else:
        return lr * tf.math.exp(-0.2)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model_vgg16.h5', save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)

# TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=test_generator,
    callbacks=[lr_callback, early_stopping, model_checkpoint, reduce_lr, tensorboard_callback]
)

# Load the best model
model = tf.keras.models.load_model('best_model_vgg16.h5')

# Save the model in HDF5 format
model.save('my_model3.h5', save_format='h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

# Confusion matrix and classification report
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = np.argmax(model.predict(test_generator), axis=1)
y_true = test_generator.classes

conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
accuracy_percentage = accuracy * 100

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel(f'Predicted labels\nAccuracy: {accuracy_percentage:.2f}%')
plt.ylabel(f'True labels\nAccuracy: {accuracy_percentage:.2f}%')
plt.title(f'Confusion Matrix\nAccuracy: {accuracy_percentage:.2f}%')
plt.show()

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

predicted_class_name = classes[y_pred[3]]
print("Predicted class name:", predicted_class_name)

classification_rep = classification_report(y_true, y_pred, target_names=classes)
print("Classification Report:\n", classification_rep)


Found 2220 images belonging to 8 classes.
Found 947 images belonging to 8 classes.
Epoch 1/100
70/70 [==============================] - ETA: 0s - loss: 61.1873 - accuracy: 0.1622 

c:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


70/70 [==============================] - 2441s 35s/step - loss: 61.1873 - accuracy: 0.1622 - val_loss: 60.0400 - val_accuracy: 0.2714 - lr: 1.0000e-05
Epoch 2/100
70/70 [==============================] - 2424s 35s/step - loss: 60.4133 - accuracy: 0.2532 - val_loss: 59.2727 - val_accuracy: 0.4562 - lr: 1.0000e-05
Epoch 3/100
70/70 [==============================] - 2387s 34s/step - loss: 59.8304 - accuracy: 0.3302 - val_loss: 58.9416 - val_accuracy: 0.4646 - lr: 1.0000e-05
Epoch 4/100
70/70 [==============================] - 2393s 34s/step - loss: 59.3195 - accuracy: 0.4014 - val_loss: 58.5551 - val_accuracy: 0.5153 - lr: 1.0000e-05
Epoch 5/100
70/70 [==============================] - 2399s 34s/step - loss: 59.0014 - accuracy: 0.4324 - val_loss: 58.4044 - val_accuracy: 0.5248 - lr: 1.0000e-05
Epoch 6/100
70/70 [==============================] - 2398s 34s/step - loss: 58.5650 - accuracy: 0.4896 - val_loss: 58.2601 - val_accuracy: 0.5153 - lr: 1.0000e-05
Epoch 7/100
70/70 [===============